# Convert themes into SITREP

In [19]:
!pip install vertexai

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 34.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.7/333.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 49.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6


In [7]:
import pandas as pd

folder_path = "./data/1714712230"
themes_l1 = pd.read_csv(f'{folder_path}/themes_l1.csv')

In [8]:
themes_l1.describe()

,title,detail
count,8,8
unique,8,8
top,Assassination of Russian defector Kuzminov in ...,The post discusses the assassination of Vladle...
freq,1,1


In [6]:
themes_l1

,title,detail
0,Assassination of Russian defector Kuzminov in ...,The post discusses the assassination of Vladle...
1,Stalemate in Semyonovka and Berdychi amid dron...,A military expert named Yuri Kotenok is quoted...
2,Russian influencer marketing on social media,One of the posts showcases an advertisement by...
3,Civilian participation in supporting Russian m...,"The post describes how teachers, students, and..."
4,Foiled terrorist plot in Dagestan,The National Anti-Terrorism Committee (NAC) of...
5,Controversy over prisoner exchanges between Ru...,The post discusses alleged provocations by Ukr...
6,Coastal defenses on the Kinburn Spit,Soldiers of the Dnieper Group of Russian troop...
7,Former US Secretary Kissinger's advice to Zele...,"According to Oleg Soskin, a former adviser to ..."


In [28]:
from anthropic import AnthropicVertex
PROJECT_ID = 'auspicious-lead-386121'
REGION = 'us-central1'

claude_client = AnthropicVertex(project_id=PROJECT_ID, region=REGION)


In [29]:
summarization_prompt = """You are an AI assistant analyzing Russian and Ukrainian social media posts.
The posts are already grouped into several themes.

I will provide you with a list of themes each with a <title> and <detail>.

Here are the requirements:
  1. Combine the related themes with a new <title> and <detail>.
  2. List all the combined themes. And, wrap them inside <combined_theme> tags.
    
"""

import re
from retry import retry

@retry(delay=10, tries=4, backoff=2)
def summarize(themes):
    prompt = f"""{summarization_prompt}
Here are the themes:
{themes}
Now combine the above themes.
    """
    # print(prompt)
    
    message = claude_client.messages.create(
        model="claude-3-sonnet@20240229",
        max_tokens=4096,
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )
    
    print(message.content[0].text)
    
    return message.content[0].text

In [30]:
themes = ""
for j in range(0, len(themes_l1)):
        row = themes_l1.iloc[j]
        themes += f"""
<theme>
<title>
{row['title']}
</title>
<detail>
{row['detail']}
</detail>
</theme>
""" 

In [31]:
summarized_themes = summarize(themes)

Here are the combined themes wrapped in <combined_theme> tags:

<combined_theme>
<title>Security Operations and Threats</title>
<detail>
This combined theme covers various security operations and threats faced by Russia and Ukraine. It includes the assassination of a Russian defector in Spain, a foiled terrorist plot in Dagestan where weapons and explosives were found, and the coastal defenses set up by Russian troops on the Kinburn Spit to counter potential Ukrainian landing forces. It also mentions the stalemate in Semyonovka and Berdychi, where Ukrainian forces are relying heavily on drone operations to sustain their defense against the Russian offensive with precision-guided munitions.
</detail>
</combined_theme>

<combined_theme>
<title>Societal Participation and Propaganda</title>
<detail>
This theme combines the aspects of civilian participation in supporting Russian military operations, such as weaving camouflage nets at schools, and the Russian influencer marketing campaign by

In [36]:
pattern = r'<combined_theme>((.|\n)*?)<\/combined_theme>'

# Find all matches
matches = re.findall(pattern, summarized_themes)  


In [37]:
themes_l2 = []

for match in matches:
    title_matches = re.findall(r'<title>((.|\n)*?)<\/title>', match[0])
    detail_matches = re.findall(r'<detail>((.|\n)*?)<\/detail>', match[0])
    # search_query_matches = re.findall(r'<search_query>((.|\n)*?)<\/search_query>', match[0])
    
    themes_l2.append({
        'title': title_matches[0][0].strip(),
        'detail': detail_matches[0][0].strip(),
        # 'search_query': search_query_matches[0][0],
        
    })

In [40]:
summarized_themes

"Here are the combined themes wrapped in <combined_theme> tags:\n\n<combined_theme>\n<title>Security Operations and Threats</title>\n<detail>\nThis combined theme covers various security operations and threats faced by Russia and Ukraine. It includes the assassination of a Russian defector in Spain, a foiled terrorist plot in Dagestan where weapons and explosives were found, and the coastal defenses set up by Russian troops on the Kinburn Spit to counter potential Ukrainian landing forces. It also mentions the stalemate in Semyonovka and Berdychi, where Ukrainian forces are relying heavily on drone operations to sustain their defense against the Russian offensive with precision-guided munitions.\n</detail>\n</combined_theme>\n\n<combined_theme>\n<title>Societal Participation and Propaganda</title>\n<detail>\nThis theme combines the aspects of civilian participation in supporting Russian military operations, such as weaving camouflage nets at schools, and the Russian influencer marketin

In [5]:
from vertexai.preview.language_models import TextEmbeddingModel, TextEmbeddingInput
model = TextEmbeddingModel.from_pretrained("text-embedding-preview-0409")

kwargs = (
        dict(output_dimensionality=256)
    )
for j in range(0, len(themes_l1)):
    text_input = TextEmbeddingInput(text="Dinner in New York City", task_type='SEMANTIC_SIMILARITY')

        row = themes_l1.iloc[j]
        
embeddings = model.get_embeddings([text_input], **kwargs)
for embedding in embeddings:
  vector = embedding.values
  print(len(vector))

256


In [1]:
from sklearn.cluster import KMeans
import numpy as np

# Sample data (replace this with your vectors)
data = np.array([[1, 2], [1.5, 1.8], [5, 8], [8, 8], [1, 0.6], [9, 11]])

# Define number of clusters
k = 2

# Initialize KMeans
kmeans = KMeans(n_clusters=k)

# Fit the data
kmeans.fit(data)

# Get the cluster centers
centroids = kmeans.cluster_centers_

# Get the labels for each data point
labels = kmeans.labels_

print("Cluster centers:", centroids)
print("Labels:", labels)


/Users/shah/opt/anaconda3/envs/sagemaker/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Cluster centers: [[1.16666667 1.46666667]
 [7.33333333 9.        ]]
Labels: [0 0 1 1 0 1]


In [41]:
news_data = pd.read_csv('~/Downloads/primer_hackathon_data_russia.csv')

In [42]:
news_data

,Source Name,Date,Original URL,Title,Content
0,RT International,2023-01-06T17:01:34+00:00,NaN,Nuclear Ukraine? Amid 'concerns' over alleged ...,"Kiev is capable of building an atomic device, ..."
1,RT International,2023-01-06T17:01:28+00:00,NaN,Latest saboteur attack foiled in Russia – media,The Federal Security Service has foiled a sabo...
2,RT International,2023-01-06T17:01:24+00:00,NaN,Russia blasts Germany’s ‘cynical’ Christmas move,Berlin announced a new batch of military suppo...
3,RT International,2023-01-06T17:01:18+00:00,NaN,Russian officials reveal status of FIFA ‘legac...,Global football governing body FIFA is on sche...
4,RT International,2023-01-06T17:01:12+00:00,NaN,Tesla forced to slash prices in China – Reuters,Tesla has reduced prices for its vehicles in C...
...,...,...,...,...,...
15505,Novaya Gazeta Europe,2023-12-27T20:26:56+00:00,NaN,Supreme Court rejects appeal by Yekaterina Dun...,Russia’s Supreme Court has rejected an appeal ...
15506,Novaya Gazeta Europe,2023-12-27T19:39:19+00:00,NaN,"Foreign agent Friday: Christo Grozev, Elvira V...",Russia’s Ministry of Justice had made its week...
15507,Novaya Gazeta Europe,2023-12-27T00:38:54+00:00,NaN,Ukraine destroys Russian landing ship in annex...,The Armed Forces of Ukraine (AFU) has said it ...
15508,Novaya Gazeta Europe,2023-12-26T19:53:30+00:00,NaN,Adam Kadyrov given honorary post in battalion ...,"Adam Kadyrov, the 16-year-old son of Ramzan Ka..."


In [44]:
print(news_data['Content'][0])

Kiev is capable of building an atomic device, and its leaders often outline such thoughts
Last year, Western media and high-ranking politicians actively discussed the possibility of Russian troops using atomic weapons in Ukraine. There has even been speculation on the likelihood of a nuclear war breaking out. However, it could be said that the risk is probably a lot higher on the other side of the barricades.
Ukraine’s Atomic History
Ukraine was a nuclear state after the collapse of the USSR, when 1,700 active atomic warheads remained in the country. Its politicians of that time had the prudence to abandon this status. The weapons were taken to Russia under international control, and their means of delivery were destroyed. Ukraine’s missile silos, with the exception of one which is now a museum near Kiev, were blown up, while its strategic bombers capable of carrying nuclear weapons were either transferred to Russia or destroyed.
Despite this, there were still many nuclear specialists 